In [13]:
import pandas as pd
import numpy as np

In [7]:
# Importing data
sgrecipient = pd.read_csv("../data/trade-register-sg-recipient.csv", header = 10, encoding= "ISO-8859-1")
sgsupplier = pd.read_csv("../data/trade-register-sg-supplier.csv", header = 10, encoding= "ISO-8859-1")

In [8]:
# Removing empty columns
sgrecipient = sgrecipient.drop(sgrecipient.columns[[3, 5, 9]], axis=1)
sgsupplier = sgsupplier.drop(sgsupplier.columns[[3, 5, 9]], axis=1)

In [9]:
# Renaming columns
sgrecipient.rename(columns = {"Recipient":"recipient", "Supplier":"supplier", "Year of order":"year", "SIPRI TIV for total order":"value"}, inplace = True)
sgrecipient = sgrecipient.iloc[:, [0, 1, 2, 11]]

sgsupplier.rename(columns = {"Recipient":"recipient", "Supplier":"supplier", "Year of order":"year", "SIPRI TIV for total order":"value"}, inplace = True)
sgsupplier = sgsupplier.iloc[:, [0, 1, 2, 11]]

In [10]:
# Standardising country name
sgrecipient['supplier'] = sgrecipient['supplier'].replace('United States', 'United States of America')
sgsupplier['recipient'] = sgsupplier['recipient'].replace({
    'United States': 'United States of America',
    'UAE': 'United Arab Emirates'
})

In [11]:
# Merge data set
arms = pd.merge(
    sgsupplier.rename(columns={'supplier': 'country1', 'recipient': 'country2', 'value': 'supplied'}), 
    sgrecipient.rename(columns={'supplier': 'country2', 'recipient': 'country1', 'value': 'received'}), 
    how='outer', 
    left_on=['country1', 'country2', 'year'], 
    right_on=['country1', 'country2', 'year']
)

In [14]:
# Convert into dummy variable
arms['total'] = arms['supplied'].fillna(0) + arms['received'].fillna(0)
arms['total'] = np.where(
    arms['total'].fillna(0) > 0,
    1,
    0
)

arms.rename(columns={'total': 'arms'}, inplace=True)

In [16]:
# Export
arms.to_csv("../data/arms.csv")